# Übung 3 - Vorverarbeitung und 3D-Visualisierung - MUSTERLÖSUNG
***

**Name:** Efim Shliamin
<br>
**Matr.-Nr.:** 573270

---

## Bearbeitungszeitraum

**Bearbeitungsbegin:** Montag, 19.12.2022
<br>
**Abgabe:** Fr, 10.02.2023, 23:59 Uhr
---

## Aufgabenbeschreibung

Die Ergebnisse von tomographischen Bildgebungsverfahren (z.B. Computertomographie, Magnetresonanztomographie oder Positronen-Emissionstomographie) sind Stapel von z.T. mehreren hundert Schnitt-/Schichtbildern primär in der Transversalebene.
Aus diesen *Originalbildern* können weitere Ebenen für eine Visualisierung berechnet werden. Der Bildstapel einer Ebene kann jedoch auch Basis einer 3D-Visualisierung sein.

Innerhalb dieser Übung sollen zwei Ziele erreicht werden:
1. Verarbeitung der einzelnen Schichtbilder als Vorbereitung einer 3D-Rekonstruktion
2. Rekonstruktion und Visualisierung eines 3D-Modells auf Basis eines Bildstapels 

Der in dieser Übung zu verwendenden Bildstapel entstammt folgendem Link: <https://mri.radiology.uiowa.edu/VHDicom/VHFCT1mm/VHF-Head.tar.gz>

**Wichtig:**
- Der oben genannte Link ist zum Zeitpunkt der Ausgabe der Übungsaufgabe nicht erreichbar!
- Verwenden Sie den Link in Aufgabe 2 aus Moodle! 

Der Datensatz ist den *CT Datasets (Visible Female CT Datasets)* des *Visible Human Project* (<https://www.nlm.nih.gov/research/visible/visible_human.html>) entnommen.

### Wichtige Hinweise zur Übung

- Für die Realisierung von **Aufgabe 4** sind externe Bibliotheken sowie andere Programmiersprachen (Code außerhalb des Notebooks) ausdrücklich zugelassen.
- Sollten Sie Bibliotheken verwenden, die sich nicht mit `pip` intallieren lassen bzw. externe Abhängigkeiten haben (z.B. VTK) oder eine andere Programmiersprache verwenden müssen Sie Ihre Lösung vorstellen:
- Abgabe mit externen Komponenten:
    - Zip-Archiv mit
        - Notebook
        - externer Quellcode
        - Bild(er) Ihrer 3D-Rekonstruktion
    - Vorstellung:
      - Während der Übung bzw. zu einem vereinbarten Termin **vor dem Abgabedatum**
      - Video / Screencast: Zeigen Sie, dass Ihr abgegebener Code die abgegebenen Bilder erzeugt
        - Software für Screencasts wäre z.B. OBS (Open Broadcast Software, https://obsproject.com/de)
        - Der Screencast soll über die Mediathek der HTW bereitgestellt werden
        - Video kann auf versteckt gestellt werden, Link zum Video in Ihrem Notebook, oder Textdatei der Abgabe
      - Sollte keine Vorstellung erfolgen, werden nur die über Moodle abgegebenen und mit den *Standard-Paketen* bzw. nachinstallierbaren Paketen ausführbaren Teile Ihrer Lösung bewertet.

**Generelle Hinweise zur Bearbeitung:** 

Für die Visualisierung der **Aufgaben 1. - 3.** soll das `matplotlib`-Paket verwendet werden. Alle Bilder sollen *inline* in diesem Notebook ausgegeben werden.

**Weitere Hinweise zur Abgabe**

- Füllen Sie unbedingt die erste Zelle unterhalb der Überschrift mit Name und Matr.-Nr. aus!
- Ergänzen Sie den Dateinamen des Notebooks vor der Abgabe um `_` und Ihre Matr.-Nr. (`Uebung 3 - 3D-Visualisierung_s0500000.ipynb`)
- Setzen Sie vor der Abgabe den Kernel zurück und testen Sie, ob Sie Ihr Notebook vollständig ausführen können!
- Entfernen Sie vor dem Upload alle Ausgaben aus dem Notebook!
- Der verwendete Bildstapel muss nicht abgegeben werden.

**Hinweise zur Benotung**

- Die Aufgabe wird nach dem üblichen Notenschema von 1,0 bis 5,0 bewertet.
- Diese Aufgabe wird mit 45% in der Gesamtnote der Übung gewichtet.

### Viel Erfolg!

---
---

### Aufgaben:

**1. Einlesen und Visualisieren des DICOM-Bildstapels**

Lesen Sie alle DICOM-Bilder des Daten-Verzeichnisses ein.

Visualisieren Sie den Bildstapel mit Hilfe eines interaktiven **Sliders** über den durch den Bildstapel navigiert werden kann. 

**Hinweise:**
- Für **alle Plots** dieses Notebooks gilt:
  - Colormap: `gray`
  - Ausblenden der Achsenbeschriftungen
- Verwenden Sie die Ihnen bekannte Bibliothek `pydicom` zum Einlesen der DICOM-Dateien.
- Nutzen sie das Paket `ipywidgets` zur Realisierung der interaktiven Elemente.
- Achten Sie beim Umgang mit Dateien und Verzeichnissen darauf, dass nicht immer nach Dateinamen sortierte Listen zurückgegeben werden.
- Bringen Sie den Bildstapel am Besten schon anhand der Dateinamen in eine sinnvolle Reihenfolge
  - Die Bilder innerhalb des Stapels sind von unten (Teil des oberen Brustkorbs) nach oben (Schädeldecke) über die Dateinamen sortiert (kleine Nummern liegen im Stapel unten, große Nummern oben).

In [ ]:
# Plots werden inline im Notebook angezeigt
%matplotlib inline

In [ ]:
"""
=============
GETTING READY
=============
"""

# pip install pydicom

# To perform 3D plotting, we are using the free version of plot.ly in offline mode 
# which uses WebGL to make visualization interactive. 
# plotly and scikit-image can be installed using conda:

# conda install plotly
# conda install scikit-image

In [ ]:
"""
===============
IMPORT PACKAGES
===============
"""

from pydicom import dcmread
import pydicom
from pydicom.data import get_testdata_file
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from __future__ import print_function
import os
import sys
import glob
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from scipy import ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
init_notebook_mode(connected=True) 

In [ ]:
"""
=============================================
READING AND VISUALISING THE DICOM IMAGE STACK
=============================================
"""

def navigation(index):       
    ds = dcmread("head/vhf."+str(index)+".dcm")
    arr = ds.pixel_array

    fig = plt.figure(figsize=(5, 5))
    text = fig.text(0.13, 1.0, f'Patient’s Name...: {ds.PatientName}'
                           f'\nPatient ID............: {ds.PatientID}'
                           f'\nModality..............: {ds.Modality}'
                           f'\nStudy Date.........: {ds.StudyDate}'
                           f'\nImage size..........: {ds.Rows} x {ds.Columns}'
                           f'\nPixel Spacing......: {ds.PixelSpacing}',
                ha='left', va='center', size=10)
    plt.imshow(arr, cmap="gray")
    plt.tick_params(labelbottom=False)
    plt.tick_params(labelleft=False)
    plt.show()
    
interact(navigation, index=widgets.IntSlider(min=1501, max=1734, step=1, value=1501), description='Index:', continuous_update=False)

**2. Konvertierung des DICOM-Bildstapels in Binärbilder**

Implementieren sie eine Funktion zur Konvertierung eines Bildes in das Binärformat anhand eines gegebenen Schwellenwertes.

In CT-DICOM-Bildern zeichnen sich die Bereiche des Untersuchungsobjektes über vergleichsweise hohe Signalwerte aus. In Gegensatz dazu ist der Hintergrund durch niedrige Signalwerte gekennzeichnet. Mit Hilfe der binären Konvertierung kann das Objekt vom Hintergrund getrennt werden.

Ihre Funktion soll die Pixel des Bildes anhand eines Vergleichs mit einem gegebenen Schwellenwert (Funktionsparameter) dem Hintergrund bzw. dem Objekt zuordnen:
- Pixelwert < Schwellenwert: Pixel ist Hintergrundpixel
- Pixelwert >= Schwellenwert: Pixel ist Objektpixel

Ihre Funktion soll auf den in den Dateien gespeicherten Werten arbeiten (**keine** Konvertierung in HU).

Wenden Sie Ihre Funktion auf alle Bilder des Stapels an. Wählen Sie hierzu einen Schwellenwert von **250**.

Visualisieren Sie den konvertierten Bilderstapel analog zu **1.** (der Hintergrund soll in **schwarz**, das Objekt in **weiß** dargestellt werden).

**Hinweis:**
- Wenn Sie zur Optimierung Ihrer 3D-Rekonstruktion den Schwellenwert für die Binarisierung angepasst haben, vermerken Sie dies kurz (z.B. als Kommentar bei der Definition der Konstanten: `threshold=X # Besser für 3D-Rekonstruktion in 4.`)

In [ ]:
"""
======================================================
CONVERSION OF THE DICOM IMAGE STACK INTO BINARY IMAGES
======================================================
"""

def navigation_binarized(index):       
    ds = dcmread("head/vhf."+str(index)+".dcm")
    arr = ds.pixel_array
    
    # specify a threshold 0-255
    threshold = 250

    # make all pixels < threshold black
    binarized = 1.0 * (arr > threshold)

    fig = plt.figure(figsize=(5, 5))
    text = fig.text(0.13, 1.0, f'Patient’s Name...: {ds.PatientName}'
                           f'\nPatient ID............: {ds.PatientID}'
                           f'\nModality..............: {ds.Modality}'
                           f'\nStudy Date.........: {ds.StudyDate}'
                           f'\nImage size..........: {ds.Rows} x {ds.Columns}'
                           f'\nPixel Spacing......: {ds.PixelSpacing}',
                ha='left', va='center', size=10)
    plt.imshow(binarized, cmap="gray")
    plt.tick_params(labelbottom=False)
    plt.tick_params(labelleft=False)
    plt.show()
    
interact(navigation_binarized, index=widgets.IntSlider(min=1501, max=1734, step=1, value=1501), description='Index:', continuous_update=False);

**3. Optimieren der Binärbilder**

Anhand der Visualisierung in **2.** ist zu erkennen, dass die Bilder z.T. kleine Artefakte im Hintergrund bzw. *Löcher* innerhalb des Objektes aufweisen. Der Objektrand ist teilweise sehr *ausgefranst*. Auf einigen Bildern sind Bereiche des Untersuchungstisches im Bild vorhanden.

Versuchen Sie diese *ungünstigen* Eigenschaften der Bilder auszugleichen.

Realisieren Sie zur Optimierung der Binärbilder die folgenden beiden Einzeloperationen:

1. Definieren eines **globalen** Objektbereichs über ein Rechteck.
    - Alle Pixel außerhalb dieses Objektbereiches sind automatisch Hintergrund
    - Wenden Sie den von Ihnen definierten Objektbereich auf alle Bilder des Stapels an
      - kann z.B. mittels Slicing realisiert werden
    - Beispiel:
    ```
    x_left = 60
    y_upper = 120
    width = 400
    height = 250
    ```
2. Morphologische Operationen (Erosion, Dilatation, Öffnung, Schließung) zur leichten Glättung der Objektränder, Schließung kleinerer Löcher im Objekt oder Entfernung kleinerer Artefakte
    - Achten Sie auf die Reihenfolge der Operationen und deren Kombinationsmöglichkeiten
    - Seien Sie mit diesen Operatoren vorsichtig. Bei übertriebener Anwendung gehen viele markante Konturinformnationen verloren 

Visualisieren Sie den optimierten Binärbildstapel analog zu **1.**

**Hinweise:**
- Die morphologische Operationen sind in den bereits installierten *Standard-Paketen* enthalten und sollten verwendet werden.

In [ ]:
"""
============================
OPTIMIZING THE BINARY IMAGES
============================
"""

def navigation_binarized_and_optimized(index):       
    ds = dcmread("head/vhf."+str(index)+".dcm")
    arr = ds.pixel_array
    
    # specify a threshold 0-255:
    threshold = 250

    # make all pixels < threshold black:
    binarized = 1.0 * (arr > threshold)
    
    # to portion:
    portion = binarized[20:340, 40:450]
    
    # optimizing:
    opened = ndimage.binary_opening(portion)
    eroded = ndimage.binary_erosion(opened)
    closed = ndimage.binary_closing(eroded)
    dilationed = ndimage.binary_dilation(closed)
    
    # plotting:
    fig = plt.figure(figsize=(5, 5))
    text = fig.text(0.13, 1.0, f'Patient’s Name...: {ds.PatientName}'
                           f'\nPatient ID............: {ds.PatientID}'
                           f'\nModality..............: {ds.Modality}'
                           f'\nStudy Date.........: {ds.StudyDate}'
                           f'\nImage size..........: {ds.Rows} x {ds.Columns}'
                           f'\nPixel Spacing......: {ds.PixelSpacing}',
                ha='left', va='center', size=10)
    plt.imshow(dilationed, cmap="gray")
    plt.tick_params(labelbottom=False)
    plt.tick_params(labelleft=False)
    plt.show()
    
interact(navigation_binarized_and_optimized, index=widgets.IntSlider(min=1501, max=1734, step=1, value=1501), description='Index:', continuous_update=False);

**4. 3D-Rekonstruktion**

Basierend auf den Binärbildern aus **3.** sollen Sie ein 3D-Modell des Datensatzes rekonstruieren.

Grundsätzlich stehen für eine Rekonstruktion verschiedene Ansätze zur Auswahl, u.a.:
- Triangulation der Oberfläche (Bildung von Dreiecken): [https://matplotlib.org/stable/gallery/mplot3d/trisurf3d_2.html](https://matplotlib.org/stable/gallery/mplot3d/trisurf3d_2.html)
- Anwendung des Marching-Cubes-Algorithmus [https://scikit-image.org/docs/dev/auto_examples/edges/plot_marching_cubes.html](https://scikit-image.org/docs/dev/auto_examples/edges/plot_marching_cubes.html)
- Konstruktion eines Volumenmodells aus den Objektvoxeln der gestapelten Schichten: [https://matplotlib.org/stable/gallery/mplot3d/scatter3d.html](https://matplotlib.org/stable/gallery/mplot3d/scatter3d.html)

Der Ansatz sowie die verwendeten Bibliotheken, die/den Sie verfolgen/verwenden wollen, bleibt Ihnen überlassen.

Ihr Ergebnis sollen Sie als 3D-Plot visualisieren:
- Ein *statisches* Perspektivbild (leichte Neigung auf allen Achsen) reicht hierfür aus (dies ist für einen 3D-Plot meist der Standard)
- Formatieren Sie Ihren Plot so, dass die Achsen die korrekten Abmessungen / Skalierungen aufweisen
  - Der Abstand auf der x- und y-Achse innerhalb eine CT-Scheibe beträgt 1 mm
  - Der Abstand zwischen den CT-Scheiben beträgt ebenfalls 1 mm
- Formatieren Sie Ihren Plot so, dass erkennbar ist, dass es sich um eine 3-dimensionale Darstellung handelt (z.B. über Transparenz, Gitterlinien, ...)

**Hinweise:**

- Berücksichtigen Sie die Reihenfolge der Bilder Ihres Bildstapels in Ihrer Rekonstruktion (der Kopf soll im 3D-Modell oben sein).
- Sie können zur Optimierung Ihrer 3D-Rekonstruktion den in **Aufgabe 2** verwendeten Schwellenwert für die Binarisierung anpassen. Vermerken Sie dies dort!
- Je nach verwendeter Bibliothek für die Rekonstruktion kann es möglich sein, dass die Visualisierung nicht mittels `matplotlib` im Notebook umgesetzt werden kann. Sollten Sie eine andere Bibliothek für die Visualisierung innerhalb des Notebooks verwenden, vermerken Sie dies.
- Visualisierungen mittels externer Bibliotheken müssen als Bilddatei mit abgegeben werden. Erstellen Sie hierzu ein Zip-Archiv aus Notebook, Bilddateien und Quellcode sowie ggf. einem Link auf ein Video / einen Screencast.
- Viele externe Bibliotheken bieten reichhaltige Funktionen zur Optimierung der Bilddaten vor einer 3D-Rekonstruktion an. Alternativ zur Nutzung der Ergebnisse aus **Aufgabe 3** können Sie eine komplette Vorverarbeitung mithilfe externer Bibliotheken als Input für eine 3D-Rekonstruktion nutzen.
    - **Dies ersetzt jedoch nicht die Bearbeitung und Abgabe der Aufgaben 1 - 3 innerhalb des Notebooks!**

In [ ]:
"""
==============
LOAD CT SLICES
==============
"""

# load the DICOM files
files = []
for index in range(1501, 1735):
    files.append(pydicom.dcmread("head/vhf."+str(index)+".dcm"))
    ds = dcmread("head/vhf."+str(index)+".dcm")
    print("loading: {}".format(index)) 
            

print("file count: {}".format(len(files)))

In [ ]:
"""
=======================================
PLOT AXIAL, SAGITTAL AND CORONAL IMAGES 
=======================================
"""

# pixel aspects, assuming all slices are the same
ps = files[0].PixelSpacing
ss = files[0].SliceThickness
ax_aspect = ps[1]/ps[0]
sag_aspect = ps[1]/ss
cor_aspect = ss/ps[0]

# create 3D array
img_shape = list(files[0].pixel_array.shape)
img_shape.append(len(files))
img3d = np.zeros(img_shape)

# fill 3D array with the images from the files
for i, s in enumerate(files):
    img2d = s.pixel_array
    img3d[:, :, i] = img2d

# plot 3 orthogonal slices
a1 = plt.subplot(2, 2, 1)
plt.imshow(img3d[:, :, img_shape[2]//2])
a1.set_aspect(ax_aspect)

a2 = plt.subplot(2, 2, 2)
plt.imshow(img3d[:, img_shape[1]//2, :])
a2.set_aspect(sag_aspect)

a3 = plt.subplot(2, 2, 3)
plt.imshow(img3d[img_shape[0]//2, :, :].T)
a3.set_aspect(cor_aspect)

plt.show()

In [ ]:
"""
================
SPECIFY THE PATH 
================
"""

data_path = "Head/"
output_path = working_path = "Save/"
g = glob(data_path + '/*.dcm')

# Print out the first 5 file names to verify we're in the right folder.
print ("Total of %d DICOM images.\nFirst 5 filenames:" % len(g))
print ('\n'.join(g[:5]))

In [ ]:
"""
===============
HELPER FUNCTION 
===============
"""

# load_scan will load all DICOM images from a folder into a list for manipulation:
def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [ ]:
"""
================
HOUNDSFELD UNITS 
================
"""

# The voxel values in the images are raw.  get_pixels_hu converts raw values into Houndsfeld units
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [ ]:
"""
===================
LET'S LOAD THE DATA 
===================
"""

id=0
patient = load_scan(data_path)
imgs = get_pixels_hu(patient)

In [ ]:
"""
===========================
LET'S SAVE THE NEW DATA SET 
===========================
"""

np.save(output_path + "fullimages_%d.npy" % (id), imgs)

In [ ]:
"""
===============================================================
LET'S NOW CREATE A HISTOGRAM OF ALL THE VOXEL DATA IN THE STUDY 
===============================================================
"""

file_used=output_path+"fullimages_%d.npy" % id
imgs_to_process = np.load(file_used).astype(np.float64) 

plt.hist(imgs_to_process.flatten(), bins=50, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
"""
========================
CRITIQUING THE HISTOGRAM 
========================

for reference (from Wikipedia):

SUBSTANCE ---  HU
air ---- -1000
lung --- -500
fat --- -100 to -50
water --- 0
blood --- +30 to +70
muscle --- +10 to +40
liver --- +40 to +60
bone --- +700 to +3000


---> Our histogram suggests the following:
- There is lots of air
- There's also some fat (brain)
- There's an abundance of water
- There is only a small bit of bone (seen as a tiny sliver of height between 1200-1700)

---> This observation means that we will need to do significant preprocessing 
if we want to process the 3D-visualisation of the skeleton 
because only a tiny bit of the voxels represent bone.
"""

In [ ]:
"""
=========================
DISPLAYING AN IMAGE STACK 
=========================
"""

id = 0
imgs_to_process = np.load(output_path+'fullimages_{}.npy'.format(id))

def sample_stack(stack, rows=6, cols=6, start_with=10, show_every=3):
    fig,ax = plt.subplots(rows,cols,figsize=[12,12])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
        ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()

sample_stack(imgs_to_process)

In [ ]:
"""
==========
RESAMPLING 
==========

Although we have each individual slices, it is not immediately clear how thick each slice is.
Fortunately, this is in the DICOM header.
"""

print("Slice Thickness: %f" % patient[0].SliceThickness)
print("Pixel Spacing (row, col): (%f, %f) " % (patient[0].PixelSpacing[0], patient[0].PixelSpacing[1]))

In [ ]:
"""
This means we have 1.0 mm slices, and each voxel represents 1.0 mm.

Because a CT slice is typically reconstructed at 512 x 512 voxels, 
each slice represents approximately 512 mm of data in length and width.

Using the metadata from the DICOM we can figure out the size of each voxel as the slice thickness. 
In order to display the CT in 3D isometric form (which we will do below), 
and also to compare between different scans, 
it would be useful to ensure that each slice is resampled in 1x1x1 mm pixels and slices.
"""

In [ ]:
"""
================================
SHAPE BEFORE VS AFTER RESAMPLING 
================================
"""

id = 0
imgs_to_process = np.load(output_path+'fullimages_{}.npy'.format(id))
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = map(float, ([scan[0].SliceThickness] + list(scan[0].PixelSpacing)))
    spacing = np.array(list(spacing))

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image, new_spacing

print("Shape before resampling\t", imgs_to_process.shape)
imgs_after_resamp, spacing = resample(imgs_to_process, patient, [1,1,1])
print("Shape after resampling\t", imgs_after_resamp.shape)

In [ ]:
"""
===========
3D PLOTTING 
===========

---> For kicks, we'll focus on rendering just THE BONES.
---> Create a high-quality static using 3D capability of matplotlib
---> Create a lower-quality but interactive render using plotly, which has WebGL support via JavaScript.
---> The marching cubes algorithm is used to generate a 3D mesh from the dataset. 
The plotly model will utilize a higher step_size with lower voxel threshold to avoid overwhelming the web browser.
"""

def make_mesh(image, threshold=-300, step_size=1):

    print("Transposing surface")
    p = image.transpose(2,1,0)
    
    print("Calculating surface")
    verts, faces, norm, val = measure.marching_cubes(p, threshold, step_size=step_size, allow_degenerate=True) 
    return verts, faces

def plotly_3d(verts, faces):
    x,y,z = zip(*verts) 
    print("Drawing")
    colormap=['rgb(236, 236, 212)','rgb(236, 236, 212)']
    
    fig = FF.create_trisurf(x=x,
                        y=y, 
                        z=z, 
                        plot_edges=False,
                        colormap=colormap,
                        simplices=faces,
                        backgroundcolor='rgb(64, 64, 64)',
                        title="Interactive Visualization")
    iplot(fig)

def plt_3d(verts, faces):
    print("Drawing")
    x,y,z = zip(*verts) 
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], linewidths=0.05, alpha=1)
    face_color = [1, 1, 0.9]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, max(x))
    ax.set_ylim(0, max(y))
    ax.set_zlim(0, max(z))
    ax.set_facecolor((0.7, 0.7, 0.7))
    plt.show()

In [ ]:
"""
==================
VERSION 1 - STATIC 
==================


---> takes approx. 30 seconds
"""

v, f = make_mesh(imgs_after_resamp, 350)
plt_3d(v, f)

In [ ]:
"""
=======================
VERSION 2 - INTERACTIVE 
=======================

---> takes approx. 45 seconds (please ignore all the warnings)
"""
v, f = make_mesh(imgs_after_resamp, 350, 2)
plotly_3d(v, f)